# Time-dependent Schrödinger Equation

Consider the time-dependent one-dimensional quantum harmonic oscillator defined by the Hamiltonian:

$$
H = \frac{\hat{p}^2}{2m} + \frac{\omega^2}{2m} \left(\hat{q} - q_0(t)\right)^2
$$

where:
$$
q_0(t) = \frac{t}{T}, \quad t \in [0, T].
$$

The initial state is given by:
$$
|\Psi_0\rangle = |n = 0\rangle
$$
(the ground state of the harmonic oscillator).

Note: we will use the standard convention of $\hbar = m = 1$ .

## Objectives

1. Compute the time-evolved state $(|\Psi(t)\rangle)$ for different values of $(T)$.
2. Plot:
   - The square norm of $(|\Psi(t)\rangle)$ as a function of $(q)$ at different times.
   - The average position of the particle $(\langle q(t) \rangle)$ as a function of time $(t)$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from numpy.polynomial.hermite import hermval
from scipy.special import factorial

## Solution ##

The analytical expression for the eigenfunctions for this Hamiltonian can be expressed through the Hermite polynomials as follows:

$$
Ψ_n(q; t) = \left(\frac{ω}{π}\right)^{1/4}\frac{1}{\sqrt{2^nn!}}
            \exp\left({\frac{-ω(q - q₀(t))^2}{2}}\right)
            H_n\left(\sqrt{ω} (q - q₀(t))\right)
$$

So, first, let's define those.

In [2]:
# Define Hermite polynomials
def hermite(x,n: int):
    
    # All coefficients of the Hermite polynomial set to zero except for the n-th, which is set to 1
    herm_coeff = np.zeros(n+1)
    herm_coeff[n] = 1
    
    # Compute the polynomial using the function from Scipy
    herm_pol = hermval(x,herm_coeff)
    
    return herm_pol

def decomposition(x, n, omega):
    ## Returns the eigenfunction of order n for the 1d quantum harmonic oscillator
    
    prefactor = (omega / np.pi)**(1/4) * (1 / np.sqrt(2**n * factorial(n)))
    psi = prefactor * np.exp(-omega * (x**2) / 2) * hermite(np.sqrt(omega) * x, n)
    
    # NORMALIZATION !!!!
    # dx = x[1] - x[0] # grid
    # psi_normalized = psi / np.sqrt(np.sum(np.abs(psi)**2) * dx)
    
    
    return psi

# Objects inizialitation # 

Now, let's create classes for containing our operators and variables.

In [3]:
class Param:
    ## Class for containing the parameters used in the simulation
    
    def __init__(self,
                 xmax: float,
                 Nx: int,
                 tsim: float,
                 Nt: int,
                 omega: float,
                 im_time: bool = False) -> None:
    
        self.xmax = xmax
        self.Nx = Nx
        self.tsim = tsim
        self.Nt = Nt
        self.omega = omega
        self.im_time = im_time
        
        # Derived parameters
        self.dx = 2 * xmax/ Nx
        self.dt = tsim / Nt
        self.x = np.arange(-xmax + xmax / Nx,self.dx)
        # Momentum, defined as 1/x and a prefactor from Fourier Transfrom
        self.dk = np.pi / xmax
        # Momentum grid
        self.k = np.concatenate((np.arange(0, Nx / 2), np.arange(-Nx / 2, 0))) * self.dk
        
class Operators:   
    # This class will hold the operators values
    
    def __init__(self, 
                 res: int,
                 par: Param,
                 voffset: float = 0.,
                 wfcoffset: float = 0.,
                 q0 = None,
                 n: int = 0) -> None:

        # Initialize the containers for the various operators
        self.V = np.empty(res, dtype=complex)
        self.R = np.empty(res, dtype=complex)
        self.K = np.empty(res, dtype=complex)
        self.wfc = np.empty(res, dtype=complex)
        
        
        # Time-dependent offset (default to zero)
        self.q0 = q0 or (lambda q: 0)
        
        # Initialize parameters and compute the energy
        if par is not None:
            self.init_operators(par, voffset, wfcoffset, n)
            self.compute_energy(par,)
            
    def init_operators(self, par: Param, voffset: float = 0, wfcoffset: float = 0, n: int = 0) -> None:
        
        # Initialize time-dependent offset
        q0 = self.q0(0)
        
        # Potential
        self.V = 0.5 * (par.omega ** 2) * (par.x - voffset - q0) **2
        self.wfc = decomposition(par.x - wfcoffset, n, par.omega).astype(complex)
        
        # Set the coefficient that regulates whether we're doing real or imaginary time evolution
        coeff = 1 if par.im_time else 1j    # for clarification, look at what done in the hands-on (ite_new.ipynb)
        
        ## Kinetic 
        ## We use Hausdorff approx (e^{A+B} \sim e^{A}*e^{B}+O(DeltaT^2))
        # In momentum space (kinetic part of the Hamiltonian)
        self.K = np.exp(-0.5 * (par.k **2) * par.dt * coeff)     # this is already the exponential term
        # In real space (potential part of the Hamiltonian)
        self.R = np.exp(-0.5 * self.V * par.dt * coeff)     # here U = e^{-V*t/2}e^{-K*t}e^{-v*t/2}+O{Deltat^3}   
        
    def compute_energy(self, par: Param) -> float:
    
        # Initialize the 3 wavefunctions: real space, momentum space, conjugate.
        wfc_r = self.wfc
        wfc_k = np.fft.fft(self.wfc)
        wfc_c = np.conj(self.wfc)
        
        # Compute both the real and momentum space energy terms
        E_r = wfc_c * self.R * wfc_r
        E_k = 0.5 * wfc_c * np.fft.ifft((par.k**2) * wfc_k) 
        
        # Sum over all space
        energy = (sum(E_r, E_k).real) * par.dx  # we take the real part since the energy is a real quantity by definition
        
        return energy
        

## Time evolution ##

Here we evolve our wavefunction in time, having the evolution operator (thanks Trotter-Suzuki approximation) as:
$$
\hat{U}(\Delta t) = e^{-i\hat{H}(t)\Delta t / \hbar} \\
                = e^{-i\hat{K}(t)\Delta t / \hbar} e^{-i\hat{V}(t)\Delta t / \hbar} + O(\Delta t^2) \\
                = e^{-i\hat{V}(t)\Delta t / 2\hbar} e^{-i\hat{K}(t)\Delta t / \hbar} e^{-i\hat{V}(t)\Delta t / 2\hbar} + O(\Delta t^3)
$$
The time evolution of the wavefunction for a time $t=N_t\Delta t$ is given by applying $N_t$ times the evolution operator, which equivaltes to:
$$
\Psi(x,t) = \hat{U}(\Delta t)^{N_t} \Psi(x,0)
$$
Then we move to the Fourier transform space and rewrite the evolution operator there.

In [4]:
def time_evolution(par: Param, opr: Operators) -> np.array: 
    
    # Store the density value of the wavefunction for each timestep (1st half for real, 2nd half for momentum)
    densities = np.zeros((100,2*par.Nx)) 
    potential = np.zeros((100,par.Nx))
    avg_pos = np.zeros((100,par.Nx))
    
    # Index for the gif
    ii = 0
    
    for t in range(par.Nt):
        # Apply the evolution operator Nt time = U^{Nt}
        
        # Update the time-dependent potential
        q0 = opr.q0(t * par.dt)
        opr.V = 0.5 * (par.omega ** 2) * (par.x - q0) **2
        
        # Update the real operator
        coeff = 1 if par.im_time else 1j
        opr.R = np.exp(-0.5 * opr.V * par.dt * coeff)
        
        # Apply the 'first half' of the potential part (real space)
        opr.wfc *= opr.R
        
        # Move to momentum space
        opr.wfc = np.fft.fft(opr.wfc)
        
        # Kinetic part
        opr.wfc *= opr.K
        
        # Back to real space
        opr.wfc = np.fft.ifft(opr.wfc)
        
        # 'Second half' of the potential part
        opr.wfc *= opr.R
        
        # Density |Psi|^2
        density =  np.abs(opr.wfc)**2
        density_momentum = np.abs(np.fft.fft(opr.wfc))**2   # np.fft.fft already normalizes
        
        # Renormalization for imaginary time
        if par.im_time:
            norm = np.sum(density * par.dx)
            opr.wfc /= np.sqrt(norm)
            density = np.abs(opr.wfc)**2
            
        # Save 100 snapshots
        if t % (par.num_t // 100) == 0 and ii < 100:
            # Save wfc 
            densities[ii, 0:par.num_x] = np.real(density)   # real space
            densities[ii, par.num_x:2 * par.num_x] = np.abs(np.fft.fft(opr.wfc)) ** 2   #momentum space
            
            # Save the potential
            potential[ii, :] = opr.V
            
            # Save the average position
            avg_pos[ii] = np.sum(par.x * density) * par.dx

        # Update the gif index
        ii += 1
        
    return densities, potential, avg_pos
        

## Testing ##

In [8]:
nx = 1000

voffset=0
wfcoffset=1
n=1
omega=0.5

params = Param(xmax=4, Nx=nx, tsim=50, Nt=20000, omega=omega,im_time=True)
ops = Operators(res=nx, par=params,voffset=voffset, wfcoffset=wfcoffset)
ops.init_operators(params, voffset, wfcoffset, n)


custom_wfc = np.zeros(nx, dtype=complex)
for n in range(10):
    custom_wfc += decomposition(params.x-wfcoffset, n, omega=omega).astype(complex)
from numpy.linalg import norm
print(norm(custom_wfc))

if custom_wfc is not None:
    # First ensure it is normalized
    custom_wfc /= norm(custom_wfc)
    ops.wfc = custom_wfc
    print(norm(custom_wfc))

ValueError: operands could not be broadcast together with shapes (1000,) (5,) 